In [ ]:
"""Tests of the analytic solution for approach 2 (for a neutrino of interest in a background).

Created 11 Nov 2023.
"""
import numpy as np
from numpy import pi, cross, cos, arccos


def scatter_backgrounds(a0, b, n, mu_t):
    a0, b = np.array(a0), np.array(b)

    # cos_theta = 2 * np.random.rand(n) - 1
    # theta = arccos(cos_theta)

    theta = pi/2 * np.ones(n)

    a = [a0]
    for theta_ in theta:
        a.append(scatter(a[-1], b, mu_t, theta_))

    return np.array(a)


def scatter(a, b, mu_t, theta=pi/2):
    coeff = mu_t * (1 - cos(theta))
    da = - coeff * cross(a, b) - coeff**2 * (a - b)
    return a + da

In [ ]:
mu_t = 0.1
n = 1000
a = scatter_backgrounds([0, 0, 1], [1, 0, 0], n, mu_t)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(-a[:, 2], ls=':')
plt.plot(-a[:, 1], ls=':')
plt.plot(a[:, 0], ls=':', lw=3)

for i in range(0):
    plt.axvline(i * 2*pi/mu_t, c='black')

x = np.arange(len(a))
y = np.exp(- mu_t**2 * x / 2)
plt.plot(x, y, c='red')

plt.plot((1 - np.linalg.norm(a, axis=-1)) * np.log(2))

# plt.plot(np.linalg.norm(a, axis=-1))

# plt.yscale('log')

In [ ]:
# Plain dumb math check.
from scipy.integrate import solve_ivp

a0 = np.array([0, 0, 1])
b = np.array([1, 0, 0]) * 0.3
c = 0.01

def integrand(t, a):
    return - c*np.cross(a, b) - c**2*(a - b)

sol = solve_ivp(integrand, t_span=(0, 10000), y0=a0)

plt.plot(sol.t, sol.y[2])
plt.plot(sol.t, np.exp(- c**2 * sol.t))


In [ ]:
# Plain dumb integration check.

dt = 1

def evolve(a):
    return a - dt*(c*np.cross(a, b) + c**2*(a - b) * 0)

a = [a0]
for _ in range(20000):
    a.append(evolve(a[-1]))
a = np.array(a)


plt.plot(a[:, 2])
plt.plot(np.linalg.norm(a, axis=1), c='lime')

x = np.arange(len(a))
plt.plot(x, np.exp(- c**2 * x*dt * 0), ls='--', lw=2, alpha=0.8)
plt.plot(x, np.exp(- c**2 * x*dt/(2) * 0), ls='--', lw=2, alpha=0.8)
plt.plot(x, np.exp(+ c**2 * x*dt/(2) * np.linalg.norm(b)**2), ls='--', lw=2, alpha=0.8)